In [18]:
## test different functions and modules 
import pickle
import importlib
from numba.typed import List
import constants as consts
import ctrlparams 
import time

from matplotlib import pyplot as plt
%matplotlib widget

## 1. Test the CLEDB_PREPINV module with faked synthetic data. 

In [22]:
## Import the constants and control parameter classes

importlib.reload(consts)
#consts=constants.Constants("fe-xiii_1074")

importlib.reload(ctrlparams)
params=ctrlparams.ctrlparams()


In [3]:
import CLEDB_PREPINV.CLEDB_PREPINV as prepinv  ##imports from the CLEDB_PREPINV subdirectory

In [4]:
with open('obsstokes_3dipole_hires_fullspectra.pkl','rb') as f:
    sobs1,sobs2,sobs3,sobsa,waveA,waveB = pickle.load(f)  
## arrange the two observation "files" in a simple list;un-necesary step given the shape of sobs array, but it mimicks a file/header structure.
sobs_lst=[sobs1[:,:,:,0:4],sobs1[:,:,:,4:8]]  ## standard python list; does not work well with numba
## set a proper numba typed list
sobs_in = List()                              ## this is the List object implemented by numba; the claim is that it is a faster performing object
[sobs_in.append(x) for x in sobs_lst]

[None, None]

## CLEDB_PREP: observation preprocessing timings

In [31]:
## preprocess the observation
#importlib.reload(prepinv)
start=time.time()
sobs_tot,yobs,rms,background,keyvals,sobs_totrot,aobs=prepinv.sobs_preprocess(sobs_in,params)
print("Full array:","{:4.6f}".format(time.time()-start),' TOTAL SECONDS FOR OBS. PREPROCESS') 

Full array: 34.038135  TOTAL SECONDS FOR OBS. PREPROCESS


## CLEDB_PREP: database preprocessing timings

In [6]:
##select and pre-read the database files based on the observation preprocessing
start=time.time()
db_enc,database,dbhdr=prepinv.sdb_preprocess(yobs,keyvals,params)
print("Full array:","{:4.6f}".format((time.time()-start)/14),' TOTAL SECONDS FOR 10^8 CALCULATIONS DATABASE PREPROCESS') ## /14 comes from loading 14 databases

Full array: 2.138113  TOTAL SECONDS FOR 10^8 CALCULATIONS DATABASE PREPROCESS


## 2. Test the CLEDB_PROC module with the same faked synthetic data. 

In [11]:
import CLEDB_PROC.CLEDB_PROC as procinv

## CLEDB_PROC: SPECTRO_PROC 2-line calculation timings

In [24]:
start=time.time()
importlib.reload(procinv)
specout=procinv.spectro_proc(sobs_in,sobs_tot,rms,background,keyvals,consts,params)
print("Full array:","{:4.6f}".format((time.time()-start)),' TOTAL SECONDS FOR SPECTROSCOPY PROCESSING')

Full array: 76.814932  TOTAL SECONDS FOR SPECTROSCOPY PROCESSING


In [30]:
## blos timing; keep commented


# ## fake 1 line
# ## set a proper numba typed list
# sobs_lst=[sobsa[:,:,:,0:4]]                   ## standard python lists will be deprecated by numba
# sobs_in = List()                              ## this is the List object implemented by numba; It utilizes memory in a column-like fashion.
# [sobs_in.append(x) for x in sobs_lst]         ## Numba developers claim that it is a significantly faster performing object


[None]

## CLEDB_PROC: BLOS_PROC 1-line calculation timings

In [33]:
# #importlib.reload(procinv)       ## If module is modified, reload the contents 
# start=time.time()
# blosout=procinv.blos_proc(sobs_tot,rms,keyvals,consts,params)
# print("Full array:","{:4.6f}".format((time.time()-start)),' TOTAL SECONDS FOR LOS FIELD PROCESSING')

<ipython-input-33-3fb43ef012de>:3: RuntimeWarning: invalid value encountered in double_scalars
  blosout=procinv.blos_proc(sobs_tot,rms,keyvals,consts,params)


Full array: 1.761549  TOTAL SECONDS FOR LOS FIELD PROCESSING


In [ ]:
importlib.reload(procinv)
#sobs_1rot=np.zeros((600,280,8))
#for xx in range(600):
#    for yy in range(280):
#        sobs_1rot[xx,yy,:]=sobs_totrot[xx,yy,:]/sobs_totrot[xx,yy,0]
invout,sfound=procinv.cledb_invproc(sobs_totrot,database,db_enc,yobs,aobs,rms,dbhdr,keyvals,params.nsearch,params.maxchisq,params.bcalc,params.reduced,params.verbose)

In [37]:
##Check inversion code output
plt.figure(figsize=(8, 8))
plt.imshow(invout[280:380,75:175,0,10])#,vmin=-1.5,vmax=1.5)
print(sfound[339,136,0:4,:])
print(sobs_totrot[339,136,:])

<ipython-input-37-5a39d8f79c87>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(8, 8))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 1.00000000e+00 -1.79847191e-01 -3.03860906e-02  6.34799826e-05
   1.75657578e-01 -3.08502419e-03 -5.21500906e-04  9.60061965e-06]
 [ 1.00000000e+00 -1.79847191e-01 -3.03860906e-02 -6.34799826e-05
   1.75657578e-01 -3.08502419e-03 -5.21500906e-04 -9.60061965e-06]
 [ 1.00000000e+00 -1.78157886e-01 -3.23812587e-02  7.87918352e-05
   1.74366258e-01 -1.83288209e-03 -3.33137255e-04  1.17665826e-05]
 [ 1.00000000e+00 -1.78157886e-01 -3.23812587e-02 -7.87918352e-05
   1.74366258e-01 -1.83288209e-03 -3.33137255e-04 -1.17665826e-05]]
[ 1.00000000e+00 -1.79763806e-01  1.70828613e-02  4.74984142e-04
  1.72989153e-01 -4.22201132e-03  4.01490097e-04  7.51123317e-05]


In [27]:
## do custom magfield inversion to check that all is good.
with open('obsstokes_3dipole_hires.pkl','rb') as f:
     sobs_a,yobs_a,rms1,xxl,yyl = pickle.load(f)  



In [13]:
plt.figure()
plt.imshow(aobs)
print(10*60*180*90,database[13].shape)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

9720000 (10, 60, 180, 90, 8)


## CLEDB_INVPROC/CLEDB_MATCH 2-line 1 pixel calculation timings

In [29]:
a,b=procinv.cledb_match(database[13][6,32,113,24,:],1.02,0,database[13],dbhdr,np.ones((8))*1e-5,8,params.maxchisq,params.bcalc,1,params.verbose) ##drirun 

for ii in [4,8,12,16,24]:
    start=time.time()
    for i in range(10): ##run each 10 times to average execution
        a1,b1=procinv.cledb_match(database[13][6,32,113,24,:],1.02,0,database[13],dbhdr,np.ones((8))*1e-5,ii,params.maxchisq,params.bcalc,0,params.verbose)
    print('1 PIXEL - ',"{:2d}".format(ii),' SOLUTIONS: ',"{:4.6f}".format((time.time()-start)/10),'TOTAL SECONDS FOR FULL DB SEARCH AND SORT') 
print("-----------------------------------------------------")
for ii in [4,8,12,16,24]:
    start=time.time()
    for i in range(10): ##run each 10 times to average execution
        a1,b1=procinv.cledb_match(database[13][6,32,113,24,:],1.02,0,database[13],dbhdr,np.ones((8))*1e-5,ii,params.maxchisq,params.bcalc,1,params.verbose)
    print('1 PIXEL - ',"{:2d}".format(ii),' SOLUTIONS: ',"{:4.6f}".format((time.time()-start)/10),'TOTAL SECONDS FOR REDUCED DB SEARCH AND SORT')  

1 PIXEL -   4  SOLUTIONS:  1.378482 TOTAL SECONDS FOR FULL DB SEARCH AND SORT
1 PIXEL -   8  SOLUTIONS:  1.502928 TOTAL SECONDS FOR FULL DB SEARCH AND SORT
1 PIXEL -  12  SOLUTIONS:  1.615763 TOTAL SECONDS FOR FULL DB SEARCH AND SORT
1 PIXEL -  16  SOLUTIONS:  1.734762 TOTAL SECONDS FOR FULL DB SEARCH AND SORT
1 PIXEL -  24  SOLUTIONS:  1.965352 TOTAL SECONDS FOR FULL DB SEARCH AND SORT
-----------------------------------------------------
1 PIXEL -   4  SOLUTIONS:  0.279699 TOTAL SECONDS FOR REDUCED DB SEARCH AND SORT
1 PIXEL -   8  SOLUTIONS:  0.585961 TOTAL SECONDS FOR REDUCED DB SEARCH AND SORT
1 PIXEL -  12  SOLUTIONS:  1.004150 TOTAL SECONDS FOR REDUCED DB SEARCH AND SORT
1 PIXEL -  16  SOLUTIONS:  1.367449 TOTAL SECONDS FOR REDUCED DB SEARCH AND SORT
1 PIXEL -  24  SOLUTIONS:  2.303643 TOTAL SECONDS FOR REDUCED DB SEARCH AND SORT


In [94]:
importlib.reload(procinv)
#invout,sfound=procinv.cledb_invproc(sobs_a,database,db_enc,yobs_a,aobs,rms,dbhdr,keyvals,params.nsearch,params.maxchisq,params.bcalc,params.reduced,params.verbose)


6291696 6291696 0.0
6360594 6360594 0.0
6296013 6296013 0.004951004468097
6356277 6356277 0.004951004468097
6291654 6291654 0.008549076115655
6360636 6360636 0.008549076115655
8900568 8900568 0.014300826714871
9583722 9583722 0.014300826714871
6295977 6295977 0.023756576666324
6356313 6356313 0.023756576666324
8900658 8900658 0.025857593767297
9583632 9583632 0.025857593767297
8900622 8900622 0.026062257352975
9583668 9583668 0.026062257352975
6296068 6296068 0.031144483935458
6356222 6356222 0.031144483935458
6295978 6295978 0.036280073954302
6356312 6356312 0.036280073954302
6296012 6296012 0.040029260653506
6356278 6356278 0.040029260653506
6360637 6360637 0.040967506325883
6291653 6291653 0.041548883142173
9583812 9583812 0.041870663138101
8900478 8900478 0.041870784273414
6296067 6296067 0.04287114442879
6356223 6356223 0.04287114442879
8900712 8900712 0.045160186320657
9583578 9583578 0.045160186320657
6291743 6291743 0.045943778364601
6360547 6360547 0.045943778364601
8900532 89

In [11]:
import time
start=time.time()
for i in range(10):
    a,b=procinv.cledb_match(database[13][6,32,113,24,:],1.02,0,database[13],dbhdr,np.ones((8))*1e-5,4,params.maxchisq,params.bcalc,0,params.verbose)
    c=i
print("{:4.6f}".format(time.time()-start),' TOTAL SECONDS FOR DBINVERT ')  

11.725557  TOTAL SECONDS FOR DBINVERT 


In [95]:
np.set_printoptions(linewidth=200)
#print(b)
# print(database[13][6,32,113,24,:])

# diff=(database[13][6,32,113,20:29,:]-database[13][6,32,113,24,:]) /1.e-5
# print(diff)
# diff*=diff

# print(diff)
# chisq = np.empty((diff.shape[0]))
# np.round_( np.sum( diff, axis=1),15,chisq)
# print(chisq)


# asrt = procinv.cledb_partsort(chisq,8)
# asrt2= np.argsort(chisq)[:8]
# print(asrt)
# print(asrt2)
print(a[:8,0],procinv.cledb_invparams(6,32,113,24,dbhdr[0]),procinv.cledb_params(6360594,dbhdr[0]))
dbhdr1=dbhdr[0][:]
dbhdr1[3]=90
print(dbhdr1,procinv.cledb_params(6360594,dbhdr1))
print(dbhdr[0])
print([dbhdr[0][0:2],[12]][0])
print("        Index          chi2            density           y                x              B             btheta            bhi             bx              by             bz")
print(a[:8,:])
print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print(a1[:38,:])

[592304. 586176. 592429. 592101. 592421. 592093. 586184. 586301.] 565408 (73, 37, 14, 2)
[ 1.000000e+01  6.000000e+01  1.800000e+02  9.000000e+01 -1.500000e+00  2.300000e+00 -1.510000e+00  1.510000e+00  0.000000e+00  6.283185e+00  0.000000e+00  3.141593e+00  1.000000e+00  1.074627e+04] (6, 32, 113, 24)
[ 1.000000e+01  6.000000e+01  1.800000e+02  9.000000e+01 -1.500000e+00  2.300000e+00 -1.510000e+00  1.510000e+00  0.000000e+00  6.283185e+00  0.000000e+00  3.141593e+00  1.000000e+00  1.074627e+04]
[10. 60.]
        Index          chi2            density           y                x              B             btheta            bhi             bx              by             bz
[[ 5.92304000e+05  0.00000000e+00  8.23258148e+00  1.02000000e+00  1.10050847e+00  2.17306537e+03  2.03589235e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00  2.17306537e+03]
 [ 5.86176000e+05  8.54907612e-03  8.49744725e+00  1.02000000e+00  8.95762706e-01  5.29712227e+03  4.21219106e-01  0.00000000e+00  0.00000

In [30]:
##Check inversion code output
plt.figure(figsize=(8, 8))
plt.imshow(invout[:,:,2,4],vmin=-1.5,vmax=1.5)
print(sfound[339,136,0,:])
#print(sobs_1rot[339,136,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0. 0. 0. 0. 0. 0. 0. 0.]


In [101]:
## Check spectroscopy
plt.figure(figsize=(8, 8))
plt.ticklabel_format(useOffset=False)
plt.imshow(specout[:,:,0,11])
print(specout[331,109,0,:])
print(specout2[331,109,0,:])
#print(specout[331,109,0,8]-specout[331,109,0,9]-0.09676675740797457)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 1.07451937e+03 -1.07488620e-01 -2.99862350e+01  2.14926472e+00
 -1.03812321e-02  9.85277253e-04 -3.71063845e-02  1.42352790e-02
  1.74022018e-01  8.68633618e-02  4.81287401e-03  2.25902800e-02]
[ 1.07452168e+03 -1.05176602e-01 -2.93412483e+01  2.14926472e+00
 -1.03812321e-02  9.85277253e-04 -3.71063845e-02  1.42352790e-02
  1.74690852e-01  8.73462306e-02  4.81287401e-03  2.25902800e-02]


In [92]:

# ## Simple Gaussian Function for spectra fitting
def obs_gaussfit(x,ymax,mean,sigma, offset):
# ## Simple Gaussian Function
    return ymax*np.exp(-(x-mean)**2/(2*sigma**2)) + offset
from scipy.optimize import curve_fit
gfit,gcov = curve_fit(obs_gaussfit,waveA[::-1]/10+1074.62686,sobs1[331,109,:,0],p0=[np.max(sobs1[331,109,:,0]),1074.5,0.2/2.35,0.0],maxfev=10000)
print(gfit[1],2*np.sqrt(2*np.log(2))*gfit[2])

plt.figure()
plt.plot(waveA[::-1]/10+1074.62686,sobs1[331,109,:,0],'b')
# plt.annotate('CDF', xy=(1.07450612e+03,2), xytext=(1.07450612e+03,1),arrowprops=dict(facecolor='black',arrowstyle="->"))
# plt.annotate('G', xy=(1.07451917e+03,2), xytext=(1.07451917e+03,1),arrowprops=dict(facecolor='black',arrowstyle="->"))
plt.plot(waveA[::-1]/10+1074.62686,obs_gaussfit(waveA[::-1]/10+1074.62686,np.max(sobs1[331,109,:,0]),specout2[331,109,0,0],specout2[331,109,0,8]/2.35,0.0),color="r",label="CDF")
plt.plot(waveA[::-1]/10+1074.62686,obs_gaussfit(waveA[::-1]/10+1074.62686,*gfit),"g*",label="Gauss_custom")
plt.plot(waveA[::-1]/10+1074.62686,obs_gaussfit(waveA[::-1]/10+1074.62686,np.max(sobs1[331,109,:,0]),specout[331,109,0,0],specout[331,109,0,8]/2.35,0.0),"y*",label="Gauss_auto")
plt.axvline(x=gfit[1],color="g")
plt.axvline(x=specout2[331,109,0,0],color="r")
plt.legend()
print((gfit[1]-1074.62686)*3e5/1074.62686,(specout2[331,109,0,0]-1074.62686)*3e5/1074.62686)

1074.519324444502 0.17416170397456915


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-30.020342734990486 -29.361801583709216


In [ ]:

# ## Simple Gaussian Function for spectra fitting
def obs_gaussfit(x,ymax,mean,sigma, offset):
# ## Simple Gaussian Function
    return ymax*np.exp(-(x-mean)**2/(2*sigma**2)) + offset
from scipy.optimize import curve_fit
gfit,gcov = curve_fit(obs_gaussfit,waveB/10+1079.78047,sobs1[331,109,:,4],p0=[np.max(sobs1[331,109,:,4]),1079.8,0.2/2.35,0.0],maxfev=10000)
print(gfit[1],2*np.sqrt(2*np.log(2))*gfit[2])

plt.figure()
plt.plot(waveB/10+1079.78047,sobs1[331,109,:,4],'b')
# plt.annotate('CDF', xy=(1.07450612e+03,2), xytext=(1.07450612e+03,1),arrowprops=dict(facecolor='black',arrowstyle="->"))
# plt.annotate('G', xy=(1.07451917e+03,2), xytext=(1.07451917e+03,1),arrowprops=dict(facecolor='black',arrowstyle="->"))
#plt.plot(waveB/10+1079.78047,obs_gaussfit(waveB/10+1079.78047,np.max(sobs1[331,109,:,4]),specout2[331,109,0,0],specout2[331,109,0,8]/2.35,0.0),color="r",label="CDF")
plt.plot(waveB/10+1079.78047,obs_gaussfit(waveB/10+1079.78047,*gfit),"g*",label="Gauss_custom")
plt.plot(waveB/10+1079.78047,obs_gaussfit(waveB/10+1079.78047,np.max(sobs1[331,109,:,4]),specout[331,109,1,0],specout[331,109,1,8]/2.35,0.0),"y*",label="Gauss_auto")
plt.axvline(x=gfit[1],color="g")
plt.axvline(x=specout[331,109,1,0],color="r")
plt.legend()
#print((gfit[1]-1079.78047)*3e5/1079.78047,(specout2[331,109,0,0]-1079.78047)*3e5/1079.78047)

In [24]:
##check the array
print(rms.shape)
plt.figure(figsize=(8, 8))
plt.imshow(rms[:,:,7])
plt.imshow(sobs_tot[:,:,1])
print(rms[333,138,0]/rms[333,138,:],sobs_tot[333,138,:])

(600, 280, 8)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[1.00000000e+00 4.10233436e-01 7.00886590e-03 5.10528257e-04
 1.89793421e-01 5.10227433e-02 8.49688898e-04 8.81079442e-05] [ 1.76905741e-03 -5.36222210e-04 -8.76758558e-06  6.38626216e-07
  6.24599028e-05 -1.20937501e-05 -1.97727590e-07  2.05031337e-08]


In [7]:
##check the array
print(sobs_in[0].shape)
plt.figure(figsize=(8, 8))
#plt.imshow(sobs_in[0][:,:,18,0]) ## stokes I of second line; first line is all only "1".

plt.imshow(aobs)
#plt.imshow(sobs_tot[:,:,3]-sobs_totrot[:,:,3])
# plt.plot(sobs_in[1][319,126,:,0])
# cdf=invprep.obs_cdf(sobs_in[1][319,126,:,0])
# plt.plot(cdf/cdf[-1])
# cdf[:]=cdf[:]/cdf[-1]
# l0=np.where(np.abs(cdf-0.05) == np.abs(cdf-0.05).min() )[0][0] 
# r0=np.where(np.abs(cdf-0.95) == np.abs(cdf-0.95).min() )[0][-1]
# print(l0,r0)
#sobs_in[0].shape[0:3]
fig, plots = plt.subplots(nrows=1, ncols=2, figsize=(11, 9))
plots[0].imshow(sobs_tot[:,:,2])
plots[1].imshow(sobs_totrot[:,:,2])

(600, 280, 30, 4)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …